## Getting your data!

In [1]:
import csv
def readbadcsv(name):
    f = open(name,'rt')
    reader = csv.reader(f)

    #once contents are available, I then put them in a list
    csv_list = []
    for l in reader:
        csv_list.append(l)
    f.close()
    #now pandas has no problem getting into a df
    df = pd.DataFrame(csv_list)
    del csv_list
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    return df

In [2]:
import os
import fun
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

dir = '../data/csv/'
files = os.listdir(dir)
if (len(files) < 3) :
    fun.downloadcsv()
df2 = readbadcsv(dir+files[1])
df1 = readbadcsv(dir+files[0])
print('yes')

yes


In [6]:
df3 = readbadcsv(dir+files[2])
print(df1.shape, df3.shape)
df3

MemoryError: Unable to allocate 5.05 GiB for an array with shape (835440, 812) and data type object